In [ ]:
using DifferentialEquations

INFO: Recompiling stale cache file /project/home17/mgh17/.julia/lib/v0.6/Primes.ji for module Primes.
INFO: Recompiling stale cache file /project/home17/mgh17/.julia/lib/v0.6/DistributedArrays.ji for module DistributedArrays.
INFO: Recompiling stale cache file /project/home17/mgh17/.julia/lib/v0.6/DifferentialEquations.ji for module DifferentialEquations.


In [ ]:
k0=0.005
c0=0.01
c1=0.4
c2=1
c3=0.1
c4=0.00135
e0=0.01
e1=1
e2=1
a0=0.01
a1=1
a2=5
b0=0.005
b1=0.005
b2=1
b3=1
gamma=0.01

alpha=0.05 # 0 to 0.4? governs the OCT4 over-expression

LIF = 100
I3 = 10 # 10,0.05

function Chickarmane(t,x,dx)
    dx[1] = (k0*x[2]*(c0+c1*x[1]^2+k0*x[2]+c2*LIF))/(1+(k0*x[2]*(c1*x[1]^2+k0*x[2]+c2*LIF+c3*x[3]^2))+c4*x[2]*x[4]^2) -  gamma*x[1]
    dx[2] = alpha + (e0+e1*x[2])/(1+e1*x[2]+e2*x[2]^2) - gamma*x[2]
    dx[3] = (a0+a1*x[2])/(1+a1*x[2]+a2*I3) - gamma*x[3]
    dx[4] = (b0 + b1*x[4]^2+b3*x[2])/(1+b1*x[4]^2+b2*x[2]^2+b3*x[2]) - gamma*x[4]
end

function σ_Chickarmane(t,x,dx)
    dx[1] = sqrt(abs((k0*x[2]*(c0+c1*x[1]^2+k0*x[2]+c2*LIF))/(1+(k0*x[2]*(c1*x[1]^2+k0*x[2]+c2*LIF+c3*x[3]^2))+c4*x[2]*x[4]^2) -  gamma*x[1]))
    dx[2] = sqrt(abs(alpha + (e0+e1*x[2])/(1+e1*x[2]+e2*x[2]^2) - gamma*x[2]))
    dx[3] = sqrt(abs((a0+a1*x[2])/(1+a1*x[2]+a2*I3) - gamma*x[3]))
    dx[4] = sqrt(abs((b0 + b1*x[4]^2+b3*x[2])/(1+b1*x[4]^2+b2*x[2]^2+b3*x[2]) - gamma*x[4]))
end

In [ ]:
prob_sde_Chickarmane = SDEProblem(Chickarmane,σ_Chickarmane,[100.0,100.0,100.0,0.0],(0.0,10000.0))

using Plots
plotly()

#[Nanog,Oct4-Sox2,FGF4,Gata6]

sol1 = solve(prob_sde_Chickarmane)

p1 = plot(sol1,vars=(4,1,3),xlabel="Gata6",ylabel="Nanog",zlabel="Fgf4")

plot(p1)

In [ ]:
p0 = plot(sol1,vars=(1,1),xlabel="",ylabel="Nanog")
p1 = plot(sol1,vars=(1,2),xlabel="",ylabel="Oct4-Sox2")
p2 = plot(sol1,vars=(1,3),xlabel="",ylabel="Fgf4")
p3 = plot(sol1,vars=(1,4),xlabel="Nanog",ylabel="Gata6")

p4 = plot(sol1,vars=(2,1))
p5 = plot(sol1,vars=(2,2))
p6 = plot(sol1,vars=(2,3))
p7 = plot(sol1,vars=(2,4),xlabel="Oct4-Sox2")

p8 = plot(sol1,vars=(3,1))
p9 = plot(sol1,vars=(3,2),xlabel="",ylabel="")
p10 = plot(sol1,vars=(3,3),xlabel="",ylabel="")
p11 = plot(sol1,vars=(3,4),xlabel="Fgf4",ylabel="")

p12 = plot(sol1,vars=(4,1))
p13 = plot(sol1,vars=(4,2),xlabel="",ylabel="")
p14 = plot(sol1,vars=(4,3),xlabel="",ylabel="")
p15 = plot(sol1,vars=(4,4),xlabel="Gata6",ylabel="")


plotly()
plot(p0,p4,p8,p12,p1,p5,p9,p13,p2,p6,p10,p14,p3,p7,p11,p15,legend=false)

In [ ]:
#NEEDS FIXING

#Nanog and Gata6 contour, x[1] and x[4]

h=100
Ps=zeros(h-1,h-1)
xspan=linspace(0,100,h)

total=0

for i = 0:10000
    
    x1init=rand([0.0,100.0])
    x2init=rand([0.0,100.0])
    x3init=rand([0.0,100.0])
    x4init=rand([0.0,100.0])
    prob_sde_Chickarmane = SDEProblem(Chickarmane,σ_Chickarmane,[x1init,x2init,x3init,x4init],(0.0,1000.0))
    sol = solve(prob_sde_Chickarmane)

    x1=1
    x4=1
    found1=false
    found4=false
    impossible=false
    tfinal=size(sol.x)[1]

    while(!(found1 && found4))
        if (sol.x[tfinal][1]<xspan[1] || sol.x[tfinal][1]>=xspan[h] || sol.x[tfinal][4]<xspan[1] || sol.x[tfinal][4]>=xspan[h])
            impossible=true
            break;
        end
            
                
        if !(xspan[x1]<=sol.x[tfinal][1] && sol.x[tfinal][1]<xspan[x1+1]) && !found1
            x1=x1+1
        else
            found1=true
        end
                
        if !(xspan[x4]<=sol.x[tfinal][4] && sol.x[tfinal][4]<xspan[x4+1]) && !found4
            x4=x4+1
        else
            found4=true
        end
    end
        
    if !impossible
        Ps[x1,x4]=Ps[x1,x4]+1
        total=total+1
    end
end


Ps=Ps/(total*(4/h)^2)+0.00001

U=-log.(Ps)